In [446]:
from bs4 import BeautifulSoup
import requests
import json

header = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'
}

sid = {'정치':100,
       '경제':101,
       '사회':102,
       '생활/문화':103,
       'IT/과학':105,
       '세계':104
}

result = {}

def fetch_link_list(sid):
    '''뉴스 카테고리 5페이지의 URL 목록 return'''
    cursor = 0
    link_list = []
    for _ in range(5):
        URL = f'https://news.naver.com/section/template/SECTION_ARTICLE_LIST?sid={sid}&next={cursor}'
        
        response = requests.get(URL, headers=header)
        datas = response.json()
        text = datas['renderedComponent']['SECTION_ARTICLE_LIST']
        soup = BeautifulSoup(text,'html.parser')

        # '더보기' 헤드에 있는 뉴스로 cursor 변경
        cursor = soup.select_one('div._PERSIST_META')['data-cursor']

        # cursor가 가리키는 뉴스로부터 1페이지 분량 URL 추출
        links = soup.select('a.sa_thumb_link')
        link_list += [link.attrs['href'] for link in links]

    return link_list

def fetch_article_data(article_url):
    '''뉴스의 제목, 본문을 dictionary형으로 return'''
    response = requests.get(article_url, headers=header)
    fetchs = BeautifulSoup(response.text, 'html.parser')
    title = fetchs.select_one('h2#title_area').text.strip()
    body = fetchs.select_one('article#dic_area').text.strip().replace('\n','')
    news_data = {
    'title':title,
    'body':body
}
    return news_data

# sid 딕셔너리의 각 항목에 대해 반복
for k, v in sid.items():
    sid_result = [fetch_article_data(article_url) for article_url in fetch_link_list(v)]
    result[k] = sid_result # result 딕셔너리에 키 카테고리 항목과 값을 sid_result로 추가

# 수집한 데이터를 JSON 파일(news_crawl_bs.json)로 저장
with open('news_crawl_bs.json', 'w', encoding='utf-8') as file:
    json.dump(result, file, indent=4, ensure_ascii=False)